# Agentic SQL Transform with LangGraph

This notebook prototypes an agent that generates and refines DuckDB SQL to transform raw input data into a target schema.

We start by building and testing the **DuckDB runner + diff logic** (no LLM yet), then add the LangGraph agent loop.


## Step 1: Environment Setup & Imports


In [ ]:
import duckdb
import pandas as pd
import os
import re
from pathlib import Path
from dataclasses import dataclass, field
from typing import Optional
from dotenv import load_dotenv

load_dotenv(override=True)

# Base path for our testdata
TESTDATA_DIR = Path("./testdata")
print(f"Testdata directory: {TESTDATA_DIR.absolute()}")
print(f"Exists: {TESTDATA_DIR.exists()}")


## Step 2: Define ScenarioConfig

A simple structure that defines what input files to load, what expected output to compare against, and other scenario metadata.


In [ ]:
@dataclass
class InputFile:
    """Represents an input file to be loaded into DuckDB."""
    path: str           # Path to the input file (CSV or JSONL)
    table_name: str     # Name of the table to create in DuckDB
    format: str = "csv" # "csv" or "jsonl"

@dataclass
class ExpectedOutput:
    """Represents the expected output file for comparison."""
    path: str           # Path to expected output CSV
    table_name: str     # Name used in final SELECT (for reference)

@dataclass
class ScenarioConfig:
    """Configuration for a single SQL transform scenario."""
    name: str
    inputs: list[InputFile]
    expected_output: ExpectedOutput
    max_iters: int = 5
    docs: list[str] = field(default_factory=list)  # Optional mapping docs


## Step 3: Define Test Scenarios from testdata/


In [ ]:
# CSV Test Scenario
csv_scenario = ScenarioConfig(
    name="csv_users_transform",
    inputs=[
        InputFile(
            path=str(TESTDATA_DIR / "csv_test" / "input_files" / "input_users.csv"),
            table_name="users_raw",
            format="csv"
        )
    ],
    expected_output=ExpectedOutput(
        path=str(TESTDATA_DIR / "csv_test" / "input_files" / "expected_users.csv"),
        table_name="users_output"
    ),
    max_iters=5
)

# JSON Test Scenario
json_scenario = ScenarioConfig(
    name="json_users_transform",
    inputs=[
        InputFile(
            path=str(TESTDATA_DIR / "json_test" / "input_files" / "input_users.jsonl"),
            table_name="users_raw",
            format="jsonl"
        )
    ],
    expected_output=ExpectedOutput(
        path=str(TESTDATA_DIR / "json_test" / "input_files" / "expected_users.csv"),
        table_name="users_output"
    ),
    max_iters=5
)

print(f"CSV Scenario: {csv_scenario.name}")
print(f"  Input: {csv_scenario.inputs[0].path}")
print(f"  Expected: {csv_scenario.expected_output.path}")
print()
print(f"JSON Scenario: {json_scenario.name}")
print(f"  Input: {json_scenario.inputs[0].path}")
print(f"  Expected: {json_scenario.expected_output.path}")


## Step 4: DuckDB Runner

1. Create an in-memory DuckDB connection
2. Load input files as tables
3. Execute the candidate SQL
4. Return the result as a DataFrame (or capture errors)


In [ ]:
@dataclass
class DuckDBRunResult:
    """Result of running SQL in DuckDB."""
    success: bool
    result_df: Optional[pd.DataFrame] = None
    error: Optional[str] = None
    tables_created: list[str] = field(default_factory=list)


def load_input_file(con: duckdb.DuckDBPyConnection, input_file: InputFile) -> str:
    """
    Load an input file into DuckDB as a table.
    Returns the SQL used for loading (for debugging).
    """
    if input_file.format == "csv":
        # Load CSV with all columns as VARCHAR (like the Go code does with raw columns)
        load_sql = f"""
            CREATE TABLE {input_file.table_name} AS 
            SELECT * FROM read_csv(
                '{input_file.path}',
                header=true,
                all_varchar=true
            )
        """
    elif input_file.format in ("json", "jsonl"):
        # Load JSONL with auto-detection
        load_sql = f"""
            CREATE TABLE {input_file.table_name} AS 
            SELECT * FROM read_json_auto(
                '{input_file.path}',
                records=true,
                sample_size=-1
            )
        """
    else:
        raise ValueError(f"Unsupported format: {input_file.format}")
    
    con.execute(load_sql)
    return load_sql


def run_duckdb(scenario: ScenarioConfig, sql: str) -> DuckDBRunResult:
    """
    Execute SQL against input files and return the result.
    
    The SQL should produce a result table/view that we can compare against expected output.
    The final statement should be a SELECT that produces the output rows.
    """
    tables_created = []
    
    try:
        # Create in-memory DuckDB connection
        con = duckdb.connect(":memory:")
        
        # Enable auto-install and auto-load of extensions (matches Go sis-config-transforms)
        con.execute("SET autoinstall_known_extensions=1;")
        con.execute("SET autoload_known_extensions=1;")
        
        # Load all input files as tables
        for input_file in scenario.inputs:
            load_input_file(con, input_file)
            tables_created.append(input_file.table_name)
        
        # Execute the candidate SQL
        result = con.execute(sql)
        result_df = result.fetchdf()
        
        con.close()
        
        return DuckDBRunResult(
            success=True,
            result_df=result_df,
            tables_created=tables_created
        )
        
    except Exception as e:
        return DuckDBRunResult(
            success=False,
            error=str(e),
            tables_created=tables_created
        )



- Load expected output CSV
- Compare columns (missing/extra)
- Compare row counts
- Sample mismatched rows
- Return a structured summary


In [ ]:
@dataclass
class DiffResult:
    """Result of comparing actual output to expected output."""
    ok: bool  # True if outputs match exactly
    error: Optional[str] = None  # Error message if DuckDB execution failed
    
    # Column differences
    missing_columns: list[str] = field(default_factory=list)
    extra_columns: list[str] = field(default_factory=list)
    
    # Row count differences
    row_count_expected: int = 0
    row_count_actual: int = 0
    
    # Sample mismatched rows (capped for context)
    # Each mismatch now includes: row_idx, expected, actual, and input_row
    sample_mismatches: list[dict] = field(default_factory=list)
    max_sample_mismatches: int = 5
    
    # Full tables for small datasets (for detailed comparison)
    expected_table: Optional[list[dict]] = None
    actual_table: Optional[list[dict]] = None
    input_table: Optional[list[dict]] = None
    
    def to_summary(self) -> str:
        """Return a human-readable summary of the diff."""
        if self.ok:
            return "Output matches expected exactly!"
        
        if self.error:
            return f"Execution error: {self.error}"
        
        parts = ["Output does not match expected:"]
        
        if self.missing_columns:
            parts.append(f"  Missing columns: {self.missing_columns}")
        if self.extra_columns:
            parts.append(f"  Extra columns: {self.extra_columns}")
        if self.row_count_expected != self.row_count_actual:
            parts.append(f"  Row count: expected {self.row_count_expected}, got {self.row_count_actual}")
        if self.sample_mismatches:
            parts.append(f"  Sample mismatches ({len(self.sample_mismatches)} shown):")
            for m in self.sample_mismatches:
                parts.append(f"    Row {m.get('row_idx', '?')}: {m}")
        
        return "\n".join(parts)


def normalize_row(row: pd.Series) -> dict[str, str]:
    normalized: dict[str, str] = {}
    for col in row.index:
        value = row[col]
        if isinstance(value, bool):
            normalized[col] = str(value).lower()
        elif pd.isna(value):
            normalized[col] = ""
        else:
            normalized[col] = str(value)
    return normalized


def compare_outputs(
    actual_df: pd.DataFrame, 
    expected_path: str, 
    input_df: Optional[pd.DataFrame] = None,
    max_samples: int = 5
) -> DiffResult:
    """
    Compare actual DataFrame output to expected CSV file.
    
    Args:
        actual_df: The output from running the SQL
        expected_path: Path to the expected output CSV
        input_df: Optional input DataFrame to include in mismatch context
        max_samples: Max number of mismatched rows to include
    
    Returns:
        DiffResult with comparison details and optional input context
    """
    # Load expected output
    expected_df = pd.read_csv(expected_path, dtype=str)
    
    # Normalize: strip whitespace from string columns
    for col in actual_df.columns:
        if actual_df[col].dtype == object:
            actual_df[col] = actual_df[col].astype(str).str.strip()
    for col in expected_df.columns:
        if expected_df[col].dtype == object:
            expected_df[col] = expected_df[col].astype(str).str.strip()
    
    result = DiffResult(
        ok=False,
        row_count_expected=len(expected_df),
        row_count_actual=len(actual_df),
        max_sample_mismatches=max_samples
    )
    
    # For small datasets, store the full tables for detailed comparison
    if len(expected_df) <= 10:
        result.expected_table = expected_df.to_dict(orient='records')
        result.actual_table = actual_df.to_dict(orient='records')
        if input_df is not None:
            result.input_table = input_df.to_dict(orient='records')
    
    # Check columns
    expected_cols = set(expected_df.columns)
    actual_cols = set(actual_df.columns)
    
    result.missing_columns = list(expected_cols - actual_cols)
    result.extra_columns = list(actual_cols - expected_cols)
    
    # If columns don't match, we can't do row-by-row comparison
    if result.missing_columns or result.extra_columns:
        return result
    
    # Sort both DataFrames by all columns for deterministic comparison
    actual_sorted = actual_df.sort_values(by=list(actual_df.columns)).reset_index(drop=True)
    expected_sorted = expected_df.sort_values(by=list(expected_df.columns)).reset_index(drop=True)
    
    # Compare row by row
    min_rows = min(len(actual_sorted), len(expected_sorted))
    mismatches = []
    
    for i in range(min_rows):
        actual_row = actual_sorted.iloc[i]
        expected_row = expected_sorted.iloc[i]
        actual_norm = normalize_row(actual_row)
        expected_norm = normalize_row(expected_row)
        
        if actual_norm != expected_norm:
            if len(mismatches) < max_samples:
                mismatch = {
                    "row_idx": i,
                    "expected": expected_norm,
                    "actual": actual_norm,
                }
                # Include input row if available (match by a key column or index)
                if input_df is not None and i < len(input_df):
                    mismatch["input"] = normalize_row(input_df.iloc[i])
                mismatches.append(mismatch)
    
    result.sample_mismatches = mismatches
    
    # Check if everything matches
    if (not result.missing_columns and 
        not result.extra_columns and 
        result.row_count_expected == result.row_count_actual and 
        not result.sample_mismatches):
        result.ok = True
    
    return result


In [ ]:
def get_output_path_for_expected(expected_path: str) -> Path:
    expected = Path(expected_path)
    output_dir = expected.parent.parent / "output_files"
    output_dir.mkdir(parents=True, exist_ok=True)

    expected_name = expected.name
    suffix = expected_name[len("expected_"):] if expected_name.startswith("expected_") else expected_name
    return output_dir / f"output_{suffix}"


def load_input_as_df(scenario: ScenarioConfig) -> Optional[pd.DataFrame]:
    """
    Load the first input file as a DataFrame for context in diffs.
    """
    if not scenario.inputs:
        return None
    
    inp = scenario.inputs[0]
    try:
        if inp.format == "csv":
            return pd.read_csv(inp.path, dtype=str)
        elif inp.format in ("json", "jsonl"):
            # For JSONL, load via DuckDB to handle nested structures
            con = duckdb.connect(":memory:")
            df = con.execute(f"SELECT * FROM read_json_auto('{inp.path}', records=true)").fetchdf()
            con.close()
            # Convert all columns to string for consistent comparison
            return df.astype(str)
    except Exception:
        return None
    return None


def run_and_compare(scenario: ScenarioConfig, sql: str) -> DiffResult:
    """
    Execute SQL against the scenario's input files and compare to expected output.
    
    This is the main "tool" function that the LangGraph agent will use.
    
    Args:
        scenario: The scenario configuration with input/expected file paths
        sql: The candidate SQL to execute (should be a SELECT that produces output rows)
    
    Returns:
        DiffResult with comparison details including input context
    """
    # Run DuckDB
    run_result = run_duckdb(scenario, sql)
    
    # If execution failed, return error result
    if not run_result.success:
        return DiffResult(
            ok=False,
            error=run_result.error
        )

    output_path = get_output_path_for_expected(scenario.expected_output.path)
    run_result.result_df.to_csv(output_path, index=False)

    actual_df = pd.read_csv(output_path, dtype=str)
    
    # Load input data for context
    input_df = load_input_as_df(scenario)
    
    # Compare outputs with input context
    return compare_outputs(
        actual_df=actual_df,
        expected_path=scenario.expected_output.path,
        input_df=input_df
    )


## Step 7: Test the DuckDB Runner + Diff with CSV Scenario

First, let's look at the input and expected data:


In [ ]:
# Preview input data for CSV scenario
print("=== CSV Scenario Input ===")
input_df = pd.read_csv(csv_scenario.inputs[0].path)
print(input_df)
print()

print("=== CSV Scenario Expected Output ===")
expected_df = pd.read_csv(csv_scenario.expected_output.path)
print(expected_df)


In [ ]:
# Test with a simple (incorrect) SQL to see the diff behavior
test_sql_wrong = """
SELECT * FROM users_raw
"""

result = run_and_compare(csv_scenario, test_sql_wrong)
print(result.to_summary())


In [ ]:
# Test with the correct SQL (translated from queries.sql)
# This is what the agent should eventually generate

correct_sql_csv = """
WITH parsed AS (
    SELECT 
        TRIM(name) AS name_clever,
        CASE 
            WHEN regexp_full_match(TRIM(age), '-?\\d+') 
            THEN CAST(TRIM(age) AS INTEGER) 
            ELSE 0 
        END AS age_clever,
        CASE 
            WHEN regexp_full_match(TRIM(height), '(\\+|\\-)?\\d+\\.\\d+') 
            THEN CAST(TRIM(height) AS FLOAT) 
            ELSE 0.0 
        END AS height_clever,
        CASE 
            WHEN regexp_full_match(TRIM(awesome), '(true|false)', 'i') 
            THEN LOWER(TRIM(awesome))
            ELSE 'false' 
        END AS awesome_clever,
        CASE 
            WHEN regexp_full_match(TRIM(bday), '\\d+-\\d+-\\d+') 
            THEN CAST(TRIM(bday) AS DATE) 
            ELSE DATE '2006-01-02' 
        END AS bday_clever
    FROM users_raw
)
SELECT * FROM parsed
"""

result = run_and_compare(csv_scenario, correct_sql_csv)
print(result.to_summary())
print()
if result.ok:
    print("CSV test passed!")
else:
    print("Details:")
    print(f"  Missing cols: {result.missing_columns}")
    print(f"  Extra cols: {result.extra_columns}")
    print(f"  Rows: expected={result.row_count_expected}, actual={result.row_count_actual}")


## Step 8: Test with JSON Scenario


In [ ]:
# Preview input data for JSON scenario
print("=== JSON Scenario Input (JSONL) ===")
with open(json_scenario.inputs[0].path) as f:
    for line in f:
        if line.strip():
            print(line.strip())
print()

print("=== JSON Scenario Expected Output ===")
expected_df = pd.read_csv(json_scenario.expected_output.path)
print(expected_df)


In [ ]:
# Let's see what the raw JSONL looks like when loaded into DuckDB
con = duckdb.connect(":memory:")
con.execute("SET autoinstall_known_extensions=1;")
con.execute("SET autoload_known_extensions=1;")
load_input_file(con, json_scenario.inputs[0])
print("Raw JSONL table schema and data:")
print(con.execute("DESCRIBE users_raw").fetchdf())
print()
print(con.execute("SELECT * FROM users_raw").fetchdf())
con.close()


In [ ]:
# Test with correct SQL for JSON scenario
# Note: name_raw is a struct with first and last fields

correct_sql_json = """
WITH parsed AS (
    SELECT 
        TRIM(name_raw.first) || ' ' || TRIM(name_raw.last) AS name_clever,
        CASE 
            WHEN regexp_full_match(TRIM(age_raw), '-?\\d+') 
            THEN CAST(TRIM(age_raw) AS INTEGER) 
            ELSE 0 
        END AS age_clever,
        CASE 
            WHEN regexp_full_match(TRIM(height_raw), '(\\+|\\-)?\\d+\\.\\d+') 
            THEN CAST(TRIM(height_raw) AS FLOAT) 
            ELSE 0.0 
        END AS height_clever,
        CASE 
            WHEN regexp_full_match(TRIM(awesome_raw), '(true|false)', 'i') 
            THEN LOWER(TRIM(awesome_raw))
            ELSE 'false' 
        END AS awesome_clever,
        CASE 
            WHEN regexp_full_match(TRIM(bday_raw), '\\d+-\\d+-\\d+') 
            THEN CAST(TRIM(bday_raw) AS DATE) 
            ELSE DATE '2006-01-02' 
        END AS bday_clever
    FROM users_raw
)
SELECT * FROM parsed
"""

result = run_and_compare(json_scenario, correct_sql_json)
print(result.to_summary())
print()
if result.ok:
    print("JSON test passed!")
else:
    print("Details:")
    print(f"  Missing cols: {result.missing_columns}")
    print(f"  Extra cols: {result.extra_columns}")
    print(f"  Rows: expected={result.row_count_expected}, actual={result.row_count_actual}")
    if result.sample_mismatches:
        print(f"  Sample mismatches: {result.sample_mismatches}")


## Step 9: Test Error Handling

Let's make sure our tool handles SQL errors gracefully.


In [ ]:
# Test with invalid SQL (non-existent table)
invalid_sql = "SELECT * FROM nonexistent_table"

result = run_and_compare(csv_scenario, invalid_sql)
print(result.to_summary())


In [ ]:
# Test with syntax error
syntax_error_sql = "SELEC * FROM users_raw"

result = run_and_compare(csv_scenario, syntax_error_sql)
print(result.to_summary())


## Summary: DuckDB Runner + Diff Logic ✓

We now have working:
1. **ScenarioConfig** - defines input files and expected output
2. **DuckDB Runner** - loads input files (CSV/JSONL) and executes SQL
3. **Diff Logic** - compares actual output to expected, returning structured summary
4. **run_and_compare()** - combined tool function ready for LangGraph

---

## Step 10: Prompt & Context Assembly for SQL Generation

Now we build the context-assembly functions that summarize schemas, samples, and diffs without overwhelming the LLM.


In [ ]:
# Quick summary of test results
print("=== DuckDB Runner + Diff Test Summary ===")
print()

# CSV test
csv_result = run_and_compare(csv_scenario, correct_sql_csv)
print(f"CSV Scenario: {'PASS' if csv_result.ok else 'FAIL'}")

# JSON test
json_result = run_and_compare(json_scenario, correct_sql_json)
print(f"JSON Scenario: {'PASS' if json_result.ok else 'FAIL'}")

# Error handling test
error_result = run_and_compare(csv_scenario, "SELECT * FROM missing")
print(f"Error Handling: {'PASS' if error_result.error is not None else 'FAIL'}")


In [ ]:
import json
from typing import Any

def extract_schema(file_path: str, file_format: str, table_name: str) -> dict:
    """
    Extract schema (column names + inferred types) from an input file.
    Returns a dict with table_name, columns, and sample rows.
    """
    con = duckdb.connect(":memory:")
    
    if file_format == "csv":
        query = f"SELECT * FROM read_csv('{file_path}', header=true, all_varchar=true) LIMIT 20"
    elif file_format in ("json", "jsonl"):
        query = f"SELECT * FROM read_json_auto('{file_path}', records=true, sample_size=-1) LIMIT 20"
    else:
        raise ValueError(f"Unsupported format: {file_format}")
    
    df = con.execute(query).fetchdf()
    
    # Get column info with types
    columns = []
    for col in df.columns:
        col_type = str(df[col].dtype)
        # Infer better type description
        if col_type == "object":
            col_type = "VARCHAR"
        columns.append({"name": col, "type": col_type})
    
    # Get sample rows (convert to list of dicts for readability)
    sample_rows = df.head(10).to_dict(orient='records')
    
    con.close()
    
    return {
        "table_name": table_name,
        "columns": columns,
        "sample_rows": sample_rows,
        "row_count": len(df)
    }


def extract_expected_schema(file_path: str) -> dict:
    """
    Extract schema from expected output CSV file.
    """
    df = pd.read_csv(file_path, dtype=str, nrows=20)
    
    columns = [{"name": col, "type": "VARCHAR"} for col in df.columns]
    sample_rows = df.head(10).to_dict(orient='records')
    
    return {
        "columns": columns,
        "sample_rows": sample_rows,
        "row_count": len(df)
    }


def format_schema_for_prompt(schema: dict, is_input: bool = True) -> str:
    """
    Format a schema dict into a concise string for the LLM prompt.
    """
    prefix = "Input" if is_input else "Expected Output"
    lines = [f"### {prefix} Table: `{schema.get('table_name', 'result')}`"]
    
    # Column info
    cols = ", ".join([f"`{c['name']}` ({c['type']})" for c in schema["columns"]])
    lines.append(f"Columns: {cols}")
    
    # Sample rows (limit to 5 for prompt brevity)
    lines.append("Sample rows (first 5):")
    for i, row in enumerate(schema["sample_rows"][:5]):
        # Truncate long values
        truncated = {k: (str(v)[:50] + "..." if len(str(v)) > 50 else str(v)) for k, v in row.items()}
        lines.append(f"  {i+1}. {json.dumps(truncated)}")
    
    return "\n".join(lines)


# Test schema extraction
print("=== Testing Schema Extraction ===\n")

# CSV input
csv_input_schema = extract_schema(
    csv_scenario.inputs[0].path,
    csv_scenario.inputs[0].format,
    csv_scenario.inputs[0].table_name
)
print(format_schema_for_prompt(csv_input_schema, is_input=True))
print()

# CSV expected output
csv_expected_schema = extract_expected_schema(csv_scenario.expected_output.path)
csv_expected_schema["table_name"] = csv_scenario.expected_output.table_name
print(format_schema_for_prompt(csv_expected_schema, is_input=False))


In [ ]:
def format_diff_for_prompt(diff: DiffResult) -> str:
    """
    Convert a DiffResult into a detailed summary for the LLM prompt.
    Includes input data context to help the LLM understand WHY values should fail validation.
    """
    if diff.ok:
        return "✓ SUCCESS: Output matches expected exactly!"
    
    if diff.error:
        # For execution errors, provide the error message
        return f"✗ EXECUTION ERROR:\n{diff.error}"
    
    # For diff mismatches, provide structured feedback
    lines = ["The previously generated SQL query is wrong - the output from your SQL does not match the expected output."]
    lines.append("")
    lines.append("")
    lines.append("Please fix the SQL query by studying the INPUT values carefully.")
    lines.append("")
    
    if diff.missing_columns:
        lines.append(f"  - Missing columns (need to add): {diff.missing_columns}")
    
    if diff.extra_columns:
        lines.append(f"  - Extra columns (should remove): {diff.extra_columns}")
    
    if diff.row_count_expected != diff.row_count_actual:
        lines.append(f"  - Row count: expected {diff.row_count_expected}, got {diff.row_count_actual}")
    
    # Show full tables for small datasets to give complete context
    if diff.input_table and diff.expected_table and diff.actual_table:
        lines.append("\n### FULL COMPARISON (input → expected vs actual):")
        for i, (inp, exp, act) in enumerate(zip(diff.input_table, diff.expected_table, diff.actual_table)):
            exp_norm = {k: str(v).strip() for k, v in exp.items()}
            act_norm = {k: str(v).strip() for k, v in act.items()}
            
            if exp_norm != act_norm:
                lines.append(f"\n  Row {i} MISMATCH:")
                lines.append(f"    INPUT:    {inp}")
                lines.append(f"    EXPECTED: {exp}")
                lines.append(f"    ACTUAL:   {act}")
                # Highlight specific differences
                for col in exp:
                    if exp_norm.get(col) != act_norm.get(col):
                        lines.append(f"    → Column `{col}`: expected '{exp_norm.get(col)}' but got '{act_norm.get(col)}'")
            else:
                lines.append(f"\n  Row {i} OK: {exp}")
    
    elif diff.sample_mismatches:
        lines.append(f"\n### VALUE MISMATCHES ({len(diff.sample_mismatches)} samples):")
        for m in diff.sample_mismatches[:5]:
            row_idx = m.get('row_idx', '?')
            expected = m.get('expected', {})
            actual = m.get('actual', {})
            input_row = m.get('input', {})
            
            lines.append(f"\n  Row {row_idx}:")
            if input_row:
                lines.append(f"    INPUT:    {input_row}")
            lines.append(f"    EXPECTED: {expected}")
            lines.append(f"    ACTUAL:   {actual}")
            
            # Show only differing columns with clear explanation
            diff_cols = [k for k in expected if expected.get(k) != actual.get(k)]
            for col in diff_cols:
                # Try to find corresponding input column
                input_col = col.replace('_clever', '').replace('_raw', '')
                input_val = input_row.get(input_col) or input_row.get(col) or input_row.get(input_col + '_raw')
                if input_val:
                    lines.append(f"    → `{col}`: input was '{input_val}' → expected '{expected.get(col)}' but got '{actual.get(col)}'")
                else:
                    lines.append(f"    → `{col}`: expected '{expected.get(col)}' but got '{actual.get(col)}'")
    
    # Add strong guidance at the end
    lines.append("")
    lines.append("=" * 60)
    lines.append("🔴 CRITICAL: Your previous SQL produced WRONG results.")
    lines.append("🔴 You MUST write a DIFFERENT SQL query that fixes the issue.")
    lines.append("🔴 DO NOT output the same SQL again - it does not work!")
    lines.append("")
    lines.append("HINTS:")
    lines.append("- Look at the INPUT values: they may contain quotes (\"), spaces, or other characters")
    lines.append("=" * 60)
    
    return "\n".join(lines)


# Test diff formatting
print("=== Testing Diff Formatting ===\n")

# Test with a wrong SQL
wrong_result = run_and_compare(csv_scenario, "SELECT * FROM users_raw")
print("Wrong SQL result:")
print(format_diff_for_prompt(wrong_result))
print()

# Test with an error
error_result = run_and_compare(csv_scenario, "SELECT * FROM missing_table")
print("Error result:")
print(format_diff_for_prompt(error_result))
print()

# Test with correct SQL
correct_result = run_and_compare(csv_scenario, correct_sql_csv)
print("Correct SQL result:")
print(format_diff_for_prompt(correct_result))


In [ ]:
def build_scenario_context(scenario: ScenarioConfig) -> str:
    """
    Build the full context for a scenario: input schemas, expected output schema.
    This is the "grounding" information that doesn't change between iterations.
    """
    lines = [f"## Scenario: {scenario.name}\n"]
    
    # Input tables
    lines.append("## INPUT TABLES\n")
    for inp in scenario.inputs:
        schema = extract_schema(inp.path, inp.format, inp.table_name)
        lines.append(format_schema_for_prompt(schema, is_input=True))
        lines.append("")
    
    # Expected output
    lines.append("## EXPECTED OUTPUT\n")
    expected_schema = extract_expected_schema(scenario.expected_output.path)
    expected_schema["table_name"] = scenario.expected_output.table_name
    lines.append(format_schema_for_prompt(expected_schema, is_input=False))
    
    return "\n".join(lines)


# System prompt for SQL generation
SQL_SYSTEM_PROMPT = """You are a DuckDB SQL expert. Your task is to write a SQL query that transforms input data into the expected output format.

## RULES:
1. Output ONLY valid DuckDB SQL - no explanations, no markdown code blocks, just the SQL.
2. The SQL should be a single SELECT statement (CTEs are allowed).
3. Use the exact column names from the expected output.
4. Handle data type conversions and validation carefully.
5. Look at the values for each column in the input data compared to those in the expected output data to understand the type and format of the data we need to enforce.
6. DuckDB uses `regexp_full_match(string, pattern)` for regex matching.
7. Do NOT use DDL statements (CREATE TABLE, etc.) - just SELECT.

## DUCKDB TIPS:
- Use TRIM() to remove whitespace
- Use CASE WHEN for conditional logic
- Use CAST() for type conversions
- For struct fields: `column_name.field_name`
- Pay attention to integer vs float values when it comes to numeric values.
"""


def build_iteration_prompt(
    scenario: ScenarioConfig,
    previous_sql: Optional[str] = None,
    last_diff: Optional[DiffResult] = None,
    evaluation: Optional[str] = None,
    iteration: int = 0
) -> str:
    """
    Build the user prompt for a specific iteration.
    Includes scenario context, previous attempt (if any), feedback, and expert evaluation.
    """
    lines = []
    
    # Add scenario context (schemas and samples)
    lines.append(build_scenario_context(scenario))
    lines.append("\n---\n")
    
    if iteration == 0:
        lines.append("## TASK")
        lines.append("Write a DuckDB SQL query that transforms the input table(s) to produce the expected output.")
        lines.append("Analyze the sample data carefully to understand the required transformations.")
    else:
        lines.append(f"## ITERATION {iteration} - REFINEMENT NEEDED")
        lines.append("⚠️ YOUR PREVIOUS SQL FAILED. You MUST fix it based on the analysis below.")
        lines.append("")
        
        if previous_sql:
            lines.append("### Your Previous (INCORRECT) SQL:")
            lines.append(f"```sql\n{previous_sql}\n```")
            lines.append("")
        
        if evaluation:
            lines.append("### 🔍 Expert Analysis of Why It Failed:")
            lines.append(evaluation)
            lines.append("")
        
        if last_diff:
            lines.append("### Detailed Failure Feedback:")
            lines.append(format_diff_for_prompt(last_diff))
            lines.append("")
        
        lines.append("### TASK")
        lines.append("Based on the expert analysis above, write a DIFFERENT SQL query that fixes the root cause.")
        lines.append("Pay special attention to the FIX section in the expert analysis.")
        lines.append("Output only the corrected SQL - do NOT repeat the same query.")
    
    return "\n".join(lines)


# Test the full prompt building
print("=== Testing Full Prompt Assembly ===\n")
print("--- First iteration prompt (truncated) ---")
first_prompt = build_iteration_prompt(csv_scenario, iteration=0)
print(first_prompt)
print()
print(f"Full prompt length: {len(first_prompt)} chars")


---

## Step 11: LangGraph State & Nodes

Now we define the LangGraph agent that iteratively generates and refines SQL:

1. **State** - Holds scenario config, current SQL, diff results, iteration count
2. **sql_generator** - Calls the LLM to generate/refine SQL
3. **duckdb_node** - Executes SQL and compares against expected output
4. **controller** - Routes to END on success or max iterations, else back to sql_generator


In [ ]:
# LangGraph imports
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# Verify OpenAI API key is available
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✓ OPENAI_API_KEY found (starts with: {api_key[:8]}...)")
else:
    print("✗ OPENAI_API_KEY not found - please set it in your .env file")


# Define the agent state
@dataclass
class HistoryEntry:
    """A single entry in the iteration history."""
    iteration: int
    sql: str
    diff_summary: str
    success: bool


class AgentState(TypedDict):
    """State for the SQL generation agent."""
    # Scenario configuration (constant through the run)
    scenario: ScenarioConfig
    
    # Current iteration
    iteration: int
    
    # Current SQL attempt
    sql: Optional[str]
    
    # Last diff result
    last_diff: Optional[DiffResult]
    
    # Expert evaluation of the failure (from evaluator node)
    evaluation: Optional[str]
    
    # History of attempts (for logging, not sent to LLM)
    history: list[HistoryEntry]
    
    # Terminal flags
    success: bool
    done: bool
    reason: Optional[str]


print("✓ AgentState defined")


In [ ]:
# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.5)


def sql_generator(state: AgentState) -> dict:
    """
    Generate or refine SQL based on the scenario and previous feedback.
    
    This node:
    1. Builds the prompt with scenario context and previous diff (if any)
    2. Calls the LLM to generate SQL
    3. Cleans the response (strips markdown, etc.)
    4. Returns updated state with new SQL
    """
    scenario = state["scenario"]
    iteration = state["iteration"]
    previous_sql = state.get("sql")
    last_diff = state.get("last_diff")
    evaluation = state.get("evaluation")
    
    # Build the prompt
    user_prompt = build_iteration_prompt(
        scenario=scenario,
        previous_sql=previous_sql,
        last_diff=last_diff,
        evaluation=evaluation,
        iteration=iteration
    )
    
    # Build messages
    messages = [
        SystemMessage(content=SQL_SYSTEM_PROMPT),
        HumanMessage(content=user_prompt)
    ]
    
    # Call the LLM
    print(f"\n{'='*60}")
    print(f"🤖 SQL Generator - Iteration {iteration}")
    print(f"{'='*60}")
    
    response = llm.invoke(messages)
    raw_sql = response.content
    
    # Clean the SQL (remove markdown code blocks if present)
    sql = raw_sql.strip()
    if sql.startswith("```"):
        # Remove markdown code fences
        lines = sql.split("\n")
        # Remove first line (```sql or ```)
        lines = lines[1:]
        # Remove last line if it's just ```
        if lines and lines[-1].strip() == "```":
            lines = lines[:-1]
        sql = "\n".join(lines)
    
    print(f"Generated SQL:\n{sql[:500]}{'...' if len(sql) > 500 else ''}")
    
    return {
        "sql": sql,
        "iteration": iteration + 1
    }


print("✓ sql_generator node defined")


In [ ]:
def format_input_for_evaluation(scenario: ScenarioConfig) -> str:
    """
    Format input data in a way that helps the evaluator understand the exact values.
    Shows the raw input with character-level detail for problematic rows.
    """
    input_df = load_input_as_df(scenario)
    if input_df is None or len(input_df) == 0:
        return "No input data available"
    
    lines = []
    lines.append("INPUT DATA (showing exact character content):")
    lines.append("")
    
    for idx, row in input_df.iterrows():
        lines.append(f"Row {idx}:")
        for col in input_df.columns:
            val = str(row[col])
            # Show the exact characters
            lines.append(f"  {col}: {repr(val)}")
            # Show what TRIM would produce
            trimmed = val.strip()
            if trimmed != val:
                lines.append(f"    → After TRIM: {repr(trimmed)}")
            # Highlight if it contains quotes
            if '"' in trimmed or "'" in trimmed:
                lines.append(f"    → ⚠️ Contains quotes!")
        lines.append("")
    
    return "\n".join(lines)


def evaluator_node(state: AgentState) -> dict:
    """
    An LLM evaluator that analyzes WHY the SQL failed and provides 
    specific guidance on how to fix it.
    
    This node only runs if there was a failure (not success).
    """
    scenario = state["scenario"]
    sql = state["sql"]
    last_diff = state.get("last_diff")
    
    # If we succeeded, no evaluation needed
    if not last_diff or last_diff.ok:
        return {"evaluation": None}
    
    print(f"\n{'='*60}")
    print(f"🔍 Evaluator - Analyzing failure")
    print(f"{'='*60}")
    
    # Build evaluator prompt
    input_context = format_input_for_evaluation(scenario)
    
    eval_prompt = f"""You are a SQL debugging expert. Analyze why this SQL query failed.

SQL QUERY:
```sql
{sql}
```

FAILURE DETAILS:
{format_diff_for_prompt(last_diff)}

{input_context}

TASK:
Analyze the failure and provide specific guidance. Focus on:

1. ROOT CAUSE: What is the fundamental reason the validation/SQL query is failing? It's important to consider the values for other rows in the input data and the expected output data to establish a pattern. This will help you understand why a specific row is failing as opposed to other rows.
2. INPUT ANALYSIS: What does the input value ACTUALLY contain compared to the expected output? Show the exact characters.
3. WHY IT FAILS: Explain why the current SQL query logic incorrectly matches or processes the input.
4. SPECIFIC FIX: Provide concrete SQL code or pattern changes needed.

Output your analysis in this format:
ROOT CAUSE: [one sentence explanation]
INPUT VALUE: [what the actual input string contains, with repr() if helpful]
WHY IT FAILS: [why the regex/validation incorrectly processes it]
FIX: [specific SQL pattern or approach - be concrete, show code if helpful]
"""

    evaluator_llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.5)
    
    messages = [
        SystemMessage(content="You are a precise SQL debugging expert. You analyze failures and provide actionable, specific fixes."),
        HumanMessage(content=eval_prompt)
    ]
    
    evaluation = evaluator_llm.invoke(messages)
    evaluation_text = evaluation.content
    
    print(f"Evaluation:\n{evaluation_text[:500]}{'...' if len(evaluation_text) > 500 else ''}")
    
    return {"evaluation": evaluation_text}


print("✓ evaluator_node defined")


In [ ]:
def duckdb_node(state: AgentState) -> dict:
    """
    Execute the current SQL and compare against expected output.
    
    This node:
    1. Runs the SQL via run_and_compare()
    2. Updates last_diff with the result
    3. Sets success=True if diff.ok
    4. Appends to history for logging
    """
    scenario = state["scenario"]
    sql = state["sql"]
    iteration = state["iteration"]
    history = state.get("history", [])
    
    print(f"\n{'='*60}")
    print(f"🔧 DuckDB Executor - Testing SQL")
    print(f"{'='*60}")
    
    # Execute and compare
    diff = run_and_compare(scenario, sql)
    
    # Format for logging
    diff_summary = format_diff_for_prompt(diff)
    print(diff_summary)
    
    # Create history entry
    entry = HistoryEntry(
        iteration=iteration,
        sql=sql,
        diff_summary=diff_summary,
        success=diff.ok
    )
    
    # Return updated state
    return {
        "last_diff": diff,
        "success": diff.ok,
        "history": history + [entry]
    }


print("✓ duckdb_node defined")


In [ ]:
def controller(state: AgentState) -> dict:
    """
    Decide whether to continue iterating or stop.
    
    Routing logic:
    - If success=True → done, route to END
    - If iteration >= max_iters → done (max iterations), route to END
    - Otherwise → route to evaluator → sql_generator
    """
    scenario = state["scenario"]
    iteration = state["iteration"]
    success = state.get("success", False)
    
    print(f"\n{'='*60}")
    print(f"🎮 Controller - Deciding next step")
    print(f"{'='*60}")
    
    if success:
        print("✓ SUCCESS! SQL matches expected output.")
        return {
            "done": True,
            "reason": "success"
        }
    
    if iteration >= scenario.max_iters:
        print(f"✗ Max iterations ({scenario.max_iters}) reached without success.")
        return {
            "done": True,
            "reason": "max_iterations"
        }
    
    print(f"→ Continuing to iteration {iteration} (max: {scenario.max_iters})")
    return {
        "done": False
    }


def route_after_controller(state: AgentState) -> Literal["sql_generator", "__end__"]:
    """
    Router function for conditional edges after controller.
    """
    if state.get("done", False):
        return END
    else:
        return "evaluator"


print("✓ controller node and router defined")


In [ ]:
# Build the LangGraph
def build_sql_agent_graph():
    """
    Assemble the StateGraph for the SQL generation agent.
    
    Graph structure:
        START → sql_generator → duckdb_node → evaluator → controller → (END or sql_generator)
    """
    # Create the graph builder with our state type
    graph_builder = StateGraph(AgentState)
    
    # Add nodes
    graph_builder.add_node("sql_generator", sql_generator)
    graph_builder.add_node("duckdb_node", duckdb_node)
    graph_builder.add_node("evaluator", evaluator_node)
    graph_builder.add_node("controller", controller)
    
    # Add edges
    graph_builder.add_edge(START, "sql_generator")
    graph_builder.add_edge("sql_generator", "duckdb_node")
    graph_builder.add_edge("duckdb_node", "controller")
    graph_builder.add_edge("evaluator", "sql_generator")
    
    # Conditional edge from controller
    graph_builder.add_conditional_edges(
        "controller",
        route_after_controller,
        {
            "evaluator": "evaluator",
            END: END
        }
    )
    
    # Compile with memory checkpointer
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)
    
    return graph


# Build the graph
sql_agent_graph = build_sql_agent_graph()

print("✓ SQL Agent Graph compiled!")
print("\nGraph structure:")
print("  START → sql_generator → duckdb_node → controller      ")
print("                   ↑                         ↓          ")
print("               evaluator                     ↓          ")
print("                   ↑                         ↓          ")
print("                   └───── (if not done) ─────┘")
print("                                             ↓          ")
print("                                           END (if done)")


In [ ]:
import uuid

def run_sql_agent(scenario: ScenarioConfig, verbose: bool = True) -> dict:
    """
    Run the SQL agent on a scenario and return the final state.
    
    Args:
        scenario: The scenario configuration
        verbose: If True, print iteration details
    
    Returns:
        The final state dict with sql, success, history, etc.
    """
    # Create initial state
    initial_state: AgentState = {
        "scenario": scenario,
        "iteration": 0,
        "sql": None,
        "last_diff": None,
        "evaluation": None,
        "history": [],
        "success": False,
        "done": False,
        "reason": None
    }
    
    # Config for this run (unique thread_id)
    config = {"configurable": {"thread_id": str(uuid.uuid4())}}
    
    if verbose:
        print(f"\n{'#'*70}")
        print(f"# Running SQL Agent on: {scenario.name}")
        print(f"# Max iterations: {scenario.max_iters}")
        print(f"{'#'*70}")
    
    # Invoke the graph
    final_state = sql_agent_graph.invoke(initial_state, config=config)
    
    if verbose:
        print(f"\n{'#'*70}")
        print(f"# FINAL RESULT")
        print(f"{'#'*70}")
        print(f"Success: {final_state.get('success', False)}")
        print(f"Iterations: {final_state.get('iteration', 0)}")
        print(f"Reason: {final_state.get('reason', 'unknown')}")
        
        if final_state.get("success"):
            print(f"\n✓ Final SQL:\n{final_state.get('sql', 'N/A')}")
    
    return final_state


print("✓ run_sql_agent helper defined")


---

## Step 12: Run the Agent on Test Scenarios

Now let's test our agentic workflow on the CSV and JSON scenarios!


In [ ]:
# Test 1: Run on CSV scenario
csv_result = run_sql_agent(csv_scenario)


In [ ]:
# Test 2: Run on JSON scenario
json_result = run_sql_agent(json_scenario)


In [ ]:
# Summary of results
print("=" * 70)
print("AGENT RUN SUMMARY")
print("=" * 70)

for name, result in [("CSV Scenario", csv_result), ("JSON Scenario", json_result)]:
    success = result.get("success", False)
    iterations = result.get("iteration", 0)
    reason = result.get("reason", "unknown")
    status = "✓ PASSED" if success else "✗ FAILED"
    
    print(f"\n{name}:")
    print(f"  Status: {status}")
    print(f"  Iterations: {iterations}")
    print(f"  Reason: {reason}")
    
    # Show history summary
    history = result.get("history", [])
    if history:
        print(f"  History ({len(history)} entries):")
        for entry in history:
            status_icon = "✓" if entry.success else "✗"
            print(f"    Iter {entry.iteration}: {status_icon}")


In [ ]:
# Examine the final generated SQL for successful runs
def show_final_sql(name: str, result: dict):
    """Display and optionally save the final SQL from a successful run."""
    if result.get("success"):
        sql = result.get("sql", "")
        print(f"\n{'='*70}")
        print(f"FINAL SQL FOR: {name}")
        print("="*70)
        print(sql)
        print("="*70)
        return sql
    else:
        print(f"\n{name}: No successful SQL generated")
        return None


# Show final SQL for both scenarios
csv_final_sql = show_final_sql("CSV Scenario", csv_result)
json_final_sql = show_final_sql("JSON Scenario", json_result)
